# Advanced Tallies in OpenMC

In this tutorial, we'll learn about more advanced tally options, focusing on different spatial representations of tallies. There are several different mechanisms by which we can define a spatial filter:

- Cells (distributed and not distributed)
- Overlaid structured meshes
- Overlaid unstructured meshes
- Functional expansions

Cell tallies directly use the regions of a cell as a filter. Overlaid meshes instead use each element in the mesh as a unique region. Finally, functional expansion tallies expand the tally collision events with a functional series representation. For example, if we want to tally the heating distribution over a cylindrical pin in a light water reactor, we might expect the power distribution to exhibit a spatial self-shielding profile, high on the periphery and low in the fuel pin center. Three different tallies could be used to capture this spatial variation.
 
<img src="filters.png" alt="drawing" width="600"/>

In [ ]:
import openmc
import numpy as np

A model that warrants advanced tallies is necessarily more complex than a pincell, so we're going to use the built-in PWR assembly model in OpenMC.

In [ ]:
model = openmc.examples.pwr_assembly()

In [ ]:
model.geometry.root_universe.plot(width=(22, 22), pixels=(500, 500))

In [ ]:
model.geometry.root_universe.bounding_box

Let's make this model finite in the vertical direction so that we'll be able to see some variation in tallies added with a $z$-component. By inspecting the model, we see that we want to change the region of cell 7, so that it is within two z-planes which bound on the top and bottom.

In [ ]:
model.geometry.root_universe.cells

In [ ]:
surfaces = model.geometry.get_all_surfaces()

In [ ]:
height = 100

cells = model.geometry.root_universe.get_all_cells()
bottom = openmc.ZPlane(z0=0, boundary_type='vacuum')
top = openmc.ZPlane(z0=height, boundary_type='vacuum')
cells[7].region = +surfaces[3] & -surfaces[4] & +surfaces[5] & -surfaces[6] & +bottom & -top 

In [ ]:
model.geometry.root_universe.cells

In [ ]:
model.geometry.root_universe.plot(width=(22, 150), pixels=(600, 600), basis='xz')

# Structured Mesh Tallies

OpenMC can tally results onto regular, rectilinear, cylindrical, spherical, and unstructured meshes. Here we'll look at how to setup a regular mesh tally and visualize it for this assembly model. To do so, we need to create a mesh filter using a `RegularMesh`. A `RegularMesh` can be defined for 1D, 2D, or 3D; here, we will set up a 3-D mesh (but we'll use only one element in the vertical direction). This would be the same as if we had set up a 2D mesh - the reason we are using a 3-D mesh is because we need a 3-D mesh in order to output results into VTK format, which we'll demo shortly.

In [ ]:
lower_left, upper_right = model.geometry.bounding_box
print(lower_left, upper_right)

In [ ]:
mesh = openmc.RegularMesh()
mesh.lower_left = lower_left
mesh.upper_right = upper_right
mesh.dimension = (50, 50, 1)

mesh_filter = openmc.MeshFilter(mesh)

Learning from our last session on tallies, we'll include a tally with all of the scores needed for determining the neutron source (we will plot our tally in conventional units for flux).

In [ ]:
mesh_tally = openmc.Tally()
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['flux', 'heating']
model.tallies = [mesh_tally]

With these tallies setup, we'll apply them and and run the model.

In [ ]:
root = model.geometry.root_universe

box = openmc.openmc.stats.Box(root.bounding_box.lower_left, root.bounding_box.upper_right)
model.settings.source = openmc.IndependentSource(space=box)

model.settings.particles = 2000
model.settings.batches = 500 
model.settings.inactive = 200

In [ ]:
statepoint = model.run(output=False)

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    mesh_tally_out = sp.get_tally(id=mesh_tally.id)

In [ ]:
mesh_flux = mesh_tally_out.get_values(scores=['flux'])
print(mesh_flux.shape)

mesh_flux = mesh_flux.reshape(mesh.dimension)
print(mesh_flux.shape)

In [ ]:
import matplotlib.pyplot as plt
img = plt.imshow(mesh_flux, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.colorbar(img, label='Flux (particle-cm/src)')

Just like in our last tutorial, we need to renormalize these flux values by (i) multiplying by the neutron source rate and (ii) dividing by the volume of each tally bin in order to get into units of neutrons/cm$^2$/s. Note that the volume chosen for normalization should always be the volume of *that* tally bin (a single mesh element in our case). To get the neutron source rate, we want the total heating over the entire domain - we have the heating tally within each mesh element, so we first need to take a sum over all those values.

Definition for source rate, $S$: 

$r \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack * \textcolor{red}{S} \left\lbrack\frac{\text{src}}{\text{s}}\right\rbrack = p \left\lbrack\frac{J}{\text{s}}\right\rbrack * \frac{1}{1.602\times10^{-19}} \left\lbrack\frac{eV}{\text{J}}\right\rbrack $

In [ ]:
r = mesh_tally_out.get_values(scores=['heating']).sum()

power = 17.34e6
neutron_source = power / 1.602e-19 / r
print(neutron_source)

For the volume normalization, we'll divide the flux values by the volume of a mesh voxel. Again, we're working with a 2D model so we'll assume an axial length of 1 cm.

In [ ]:
volume = np.prod((mesh.upper_right - mesh.lower_left) / mesh.dimension)
print(volume)

To normalize the neutron flux, we now do

$f \left\lbrack\frac{\text{particle-cm}}{\text{src}}\right\rbrack * \frac{S}{V} \left\lbrack\frac{\text{src}}{\text{s cm$^3$}}\right\rbrack \rightarrow \left\lbrack\frac{\text{particle}}{\text{cm$^2$-s}}\right\rbrack $

In [ ]:
img = plt.imshow(mesh_flux * neutron_source / volume, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.colorbar(img, label='Flux (1/cm$^2$/s)')

We can also plot the heating distribution. To normalize the volumetric heating, we now do

$q \left\lbrack\frac{\text{eV}}{\text{src}}\right\rbrack * \frac{S}{V} \left\lbrack\frac{\text{src}}{\text{s cm$^3$}}\right\rbrack * 1.602\times10^{-19} \left\lbrack\frac{\text{J}}{\text{eV}}\right\rbrack\rightarrow \left\lbrack\frac{\text{W}}{\text{cm$^3$}}\right\rbrack $

In [ ]:
mesh_heat = mesh_tally_out.get_values(scores=['heating']) * neutron_source / volume * 1.602e-19
mesh_heat = mesh_heat.reshape(mesh.dimension)

In [ ]:
img = plt.imshow(mesh_heat * 1e-3, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(img, label='Heating (kW/cm$^3$)')

### Writing to VTK

OpenMC's built-in mesh tallies (for Cartesian, cylindrical, and spherical meshes) can also be written to VTK for easier viewing (this is especially helpful for 3-D meshes). Create a dictionary with the datasets you want to write. Note that the formatting is important - you can simply pass in the tally mean array without reshaping. Because we have two scores, we need to take a slice so that we obtain a tally object but which only has the one slice for the score we want to write.

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    t = sp.get_tally(id=mesh_tally.id).get_slice(scores=['heating'])

In [ ]:
data = {'heating': t.mean}
mesh.write_data_to_vtk(filename='tally.vtk', datasets=data)

In [ ]:
!ls

We can now open this file in Paraview, a free to download program for visualization: https://www.paraview.org/. Below are the steps you take in order to view the heating data volumetrically.

<img src="paraview.png" alt="drawing" width="700"/>

### Manipulating the tally arrays

The `get_values()` method gives us an array with three dimensions: (filters, nuclides, scores). If you have multiple filters in a tally, the `get_reshaped_data()` method will give you a separate dimension for each filter. For our mesh case, this effectively gives the same thing as `get_values()` since there's only a single filter:

In [ ]:
mesh_tally_out.shape

In [ ]:
mesh_tally_out.get_reshaped_data().shape

However, there is also an `expand_dims` argument that will expand a mesh filter into multiple dimensions:

In [ ]:
mesh_data = mesh_tally_out.get_reshaped_data(expand_dims=True)
mesh_data.shape

In [ ]:
mesh_data = mesh_tally_out.get_reshaped_data(expand_dims=True).squeeze()
mesh_data.shape

Now we can index the array if we want to pull out specific values. In this case, we see two values because we have two scores.

In [ ]:
mesh_data[0, 0]

It would be easier to postprocess this data if we could extract out each score one at a time. To do so, we'll use the `get_slice` method. This method differs from the `get_values` method in that it returns an `openmc.Tally`, rather than a numpy data structure.

In [ ]:
flux_only = mesh_tally_out.get_slice(scores=['flux'])
print(type(flux_only))
flux_only.shape

In [ ]:
flux_reshaped = flux_only.get_reshaped_data(expand_dims=True).squeeze()
flux_reshaped.shape

In [ ]:
flux_reshaped[0, 0]

# Unstructured Mesh Tallies

OpenMC can also use 3-D unstructured mesh tallies, if those meshes are generated in libMesh or MOAB format. This portion of the notebook you will only be able to run if you have configured OpenMC with the appropriate mesh library support for the mesh format you would like to use. For more information, see the optional prerequisites here: https://docs.openmc.org/en/stable/usersguide/install.html#prerequisites

We will use a libMesh mesh tally here. We have generated a 3-D mesh of a pincell using meshing software (such as Cubit, Gmsh, or MOOSE's reactor module). For libMesh tallies, the estimator has to be a collision estimator because OpenMC does not yet presently collect the trackklength information passing through each cell.

In [ ]:
um = openmc.UnstructuredMesh(filename='mesh_in.e', library='libmesh')
um_filter = openmc.MeshFilter(um)
mesh_tally.filters = [um_filter]
mesh_tally.estimator = 'collision'

We will want to run with more particles to be sure that each bin in the mesh gets some hits (but we'll not use a tremendous amount since this is just intended for learning).

In [ ]:
model.settings.particles = 4000
statepoint = model.run(output=False)

When we run with a mesh tally, OpenMC will automatically write the tally results into a new output file named `tally_1.<batches>.e`, where we may have multiple output files if we have multiple separate mesh tallies. We can open this file in visualization software such as [Paraview](https://www.paraview.org/).

In [ ]:
!ls

## Distributed cells (distribcells)

So this gives us a fairly good idea of what the flux and power distributions look like in this model, but we often want to know the per-pin power generation rate -- something that is hard to post-process with the tallies above (especially because the mesh tally is not conformal to the geometry). We can use a distribcell tally to produce this information easily.

A distributed cell (distribcell) is how OpenMC stores cells in universes which are repeated in lattices. In short, each cell in OpenMC is associated with an *id* and an *instance*. If a cell is repeated multiple times throughout a geometry, that cell has the same id, but with unique instances.

First, we'll want to create a distribcell tally for the cell containing our fuel material. Based on the list of materials we loaded with the pre-built model, our fuel material has the name "Fuel". We'll use that to identify the cell we want to setup a distribcell tally for.

In [ ]:
model.materials

In [ ]:
fuel_cell = None

for cell_id, cell in model.geometry.get_all_material_cells().items():
    if cell.fill.name == 'Fuel':
        fuel_cell = cell
        
print(fuel_cell)

In [ ]:
model.geometry.determine_paths()
print(fuel_cell.num_instances)

In [ ]:
distribcell_filter = openmc.DistribcellFilter(fuel_cell)

In [ ]:
dcell_tally = openmc.Tally()
dcell_tally.filters = [distribcell_filter]
dcell_tally.scores = ['heating']
model.tallies = [dcell_tally]

In [ ]:
statepoint = model.run(output=False)

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    dcell_tally_out = sp.tallies[dcell_tally.id]
    heat = dcell_tally_out.get_values(scores=['heating'])    

heat_df = dcell_tally_out.get_pandas_dataframe()
heat_df

It's easier to visualize this tally with the `openmc-plotter`. To view,

- `openmc-plotter model.xml`
- Load in the statepoint we just generated
- Click to view the tally in the bottom of the right toolbar ("Apply Changes" to view)

## Functional Expansion Tallies

For our last advanced form of tally, we'll create a functional expansion tally (FET). A FET is an expansion of some quantity (such as a tally) in terms of an infinite sum of orthogonal polynomials. Examples of orthogonal polynomials include the

- Legendre basis (Cartesian)
- Fourier basis ($0\leq\theta\leq 2\pi$)
- Zernike basis (the unit disc)
- Bessel functions (radial coordinate in cylindrical coordinates)
- spherical harmonics basis (the surface of a unit sphere)
- ...

For example, suppose we have some quantity $f(x)$. We can write this function as an infinite series sum of coefficients multiplying the Legendre polynomials,

$f(x)=\sum_{n\ =\ 0}^\infty C_nP_n(x)$

If we operate both sides by an integral with the orthogonal polynomial, we find that an FET in OpenMC is tallying the coefficients $C_n$, and that the "score" we specify is being multiplied by the evaluation of the polynomial at the collision location. FETs have to be used with collision estimators.

$\int f(x)P_m(x)dx=\int\sum_{n\ =\ 0}^\infty C_nP_n(x)P_m(x)\rightarrow \int f(x)P_m(x)dx=C_n\underbrace{\int P_n(x)P_n(x)}_\text{unity if orthonormal}$

In [ ]:
legendre = openmc.SpatialLegendreFilter(10, 'z', 0.0, height)

In [ ]:
fet = openmc.Tally()
fet.filters = [legendre]
fet.scores = ['heating']
fet.estimator = 'collision'
model.tallies = [fet]

In [ ]:
model.settings.particles = 2000
statepoint = model.run(output=False)

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    tally = sp.get_tally(scores=['heating'])

coeffs = tally.get_values().squeeze()

In [ ]:
coeffs

In [ ]:
poly = openmc.legendre_from_expcoef(coeffs, domain=(0.0, height))

In [ ]:
xvals = np.linspace(0, height, 1000)
polyvals = [poly(x) for x in xvals]
plt.plot(xvals, polyvals)

Even better, we could explore the impact of truncating the infinite series at a finite number of points. When we choose an order $N$ polynomial, we could be missing spatial detail for orders greater than $N$.

For the Legendre series, the first few polynomials are:

- $P_0(x)=1$
- $P_1(x)=x$
- $P_2(x)=\frac{1}{2}(3x^2-1)$
- $P_3(x)=\frac{1}{2}(5x^3-3x)$
- ...

In [ ]:
for o in range(1, legendre.order):
    poly = openmc.legendre_from_expcoef(coeffs[:o], domain=(0.0, height))
    polyvals = [poly(x) for x in xvals]

    plt.plot(xvals, polyvals, label='$N={}$'.format(o))
plt.legend()

In the axial direction, the solution to the 1-group homogeneous diffusion equation gives a sine function - we therefore expect that choosing order 0 (a constant) or order 1 (a constant plus a line) will be a very poor approximation to this shape. Second order does quite a good job - we have an infinitely smooth solution (as opposed to piecewise constant cell/mesh tallies)! However, downsides of the FET are that it is not trivial to construct an orthogonal polynomial over an arbitrary domain.

To make things a little bit more interesting, we could make one of the vertical boundaries an albedo boundary. Albedos are ratios of partial currents,

$\text{albedo}=\frac{J_{in}}{J_{out}}$

In other words, by setting an albedo of 97%, a particle's importance is reduced by 3% upon re-entering.

In [ ]:
top.boundary_type='reflective'
top.albedo = 0.97

model.settings.particles = 2000
statepoint = model.run(output=False)

fig, ax = plt.subplots(2, 1, figsize=(8,8))

with openmc.StatePoint(statepoint) as sp:
    tally = sp.get_tally(scores=['heating'])

coeffs = tally.get_values().squeeze()

for o in range(1, legendre.order):
    poly = openmc.legendre_from_expcoef(coeffs[:o], domain=(0.0, height))
    polyvals = [poly(x) for x in xvals]

    ax[0].plot(xvals, polyvals, label='$N={}$'.format(o))

    if (o > 2):
        ax[1].plot(xvals, polyvals, label='$N={}$'.format(o))
    ax[1].set_xlim([80, 100])
    ax[1].set_ylim([1e6, 1.2e6])

ax[0].grid()
ax[1].grid()
plt.legend(ncol=3)